In [1]:
from pyproj import enums, database, Transformer, CRS
import pandas as pd
from shapely.geometry import Polygon, Point
import requests
import sys

## Used functions
For more information, we refer to the paper and the readme.md file.

**`getProjectionTypes(self)`:**
A simple function to obtain an overview of different types of CRS

Returns
- `List`: A list of different types of CRS.

In [13]:
def getProjectionTypes():
    return dir(enums.PJType)

**`geocode(self, string)`:**
Function that retreives a list of coordinates for a given descriptive location. This function uses the Nonminatim API for geocoding.

Parameters
- `locStr` : String, description of the spatial location.

Returns
- `refList` : List of Points (shapely.geometry), a list of coordinates for a given descriptive location.

In [ ]:
def geocode(locStr):
    url = "https://nominatim.openstreetmap.org/search"
    params = {"format": "json", "q": locStr}
    r = requests.get(url, params=params)
    locList = r.json()
    refList = []
    for loc in locList:
        refLAT, refLON = (float(loc['lat']), float(loc['lon']))
        refPoint = Point(refLON, refLAT)
        refList.append(refPoint)
    print("Obtained coordinates for %s" % locStr)
    return refList

**`getAuthorityNames(self)`:**
A simple function to obtain an overview of implemented authorities in PyProj.

Returns
- `List` : A list of authorities (string).

In [14]:
def getAuthorityNames():
    return database.get_authorities()

**`getCandidateList(self, Point, int)`:**
Function that generates a list of all CRS in the database that intersect with the given point. The selection is based on the intersection of this point with the boundaries of the "area of use" of all available CRS.

Parameters
- `refPoint` : Point (shapely.geometry), a point that contains the geographic coordinates of the estimated location of the study area (in LON, LAT).
- `buffer` : Integer, optional, optional value that represents a buffer distance around the given point. To be used when the estimated location cannot be defined with great certainty. The default is 0.

Returns
- `candidates` : List, a list of pyproj.CRS objects corresponding with the CRS intersecting the given reference point.

In [15]:
def getCandidateList(refPoint, buffer=0):
    candidates = []
    epsgList = database.get_codes('EPSG', 'PROJECTED_CRS')
    if buffer > 0 and buffer < 1:
        epsgList = database.get_codes('EPSG', 'GEOGRAPHIC_CRS')
    for epsg in epsgList:
        crs = CRS.from_epsg(epsg)
        crsBounds = crs.area_of_use.bounds
        crsBoundsPolygon = Polygon(((crsBounds[0], crsBounds[1]),
            (crsBounds[2], crsBounds[1]), (crsBounds[2], crsBounds[3]),
            (crsBounds[0], crsBounds[3])))
        if crsBoundsPolygon.intersects(refPoint):
            candidates.append(crs)
    print("Prepared list of candidate CRS for [%.3f, %.3f]" % (refPoint.y, refPoint.x))
    return candidates

**`evaluateCRSList(self, list, Point, Point)`:**
For each CRS in the list, the reprojected coordinates of the reference point are calculated. Then, the distance between the given point with unknown CRS and the reprojected reference point is calculated. This function results in a sorted Pandas DataFrame with reference to the CRS and the accompanying distance.

Parameters
- `candidates` : List: a list of pyproj.CRS objects corresponding with the CRS intersecting the given reference point.
- `givenPoint` : Point (shapely.geometry): a point with given coordinates but with an unknown CRS.
- `refPoint` : Point (shapely.geometry): a point that contains the geographic coordinates of the estimated location of the study area (in LON, LAT).

Returns
- `assessmentDf` : DataFrame (pandas), a Pandas DataFrame containing a distance for each CRS. The following keys are used:
 - `crs`: pyproj.CRS-object, the CRS itself
 - `name`: string, the name of the CRS
 - `dist`: float, the distance between the projected reference point and the given point (in m).

In [16]:
def evaluateCRSList(candidates, givenPoint, refPoint):
    assessment = []
    for candidate in candidates:
        try:
            refCRS = CRS.from_epsg(4326)
            if candidate is not refCRS:
                transformer = Transformer.from_crs(refCRS, candidate, always_xy=True)
                calcX, calcY = transformer.transform(refPoint.x, refPoint.y)
                calcPoint = Point(calcX, calcY)
            else:
                calcPoint = refPoint
            dist = givenPoint.distance(calcPoint)
            assessment.append({'crs': candidate, 'name': candidate.name, 'dist': dist})
        except:
            continue
    assessmentDf = pd.DataFrame.from_records(assessment)
    assessmentDf = assessmentDf.sort_values('dist')
    print("Calculated distances for candidate CRS for [%.3f, %.3f]" % (refPoint.y, refPoint.x))
    return assessmentDf

## The code that combines all together
The code is demonstrated for a coordinate located in Ghent, Belgium:

In [19]:
refList = geocode("Gent")
givenX, givenY = 105000, 194000

givenPoint = Point(givenX, givenY)
buffer = 1500
threshold = sys.maxsize

bestCRSdf = pd.DataFrame({"dist": threshold}, index=[0])

# FOR PYTHON 3.9
# refPointIter = iter(refList)
# while (refPoint := next(refPointIter, None)) is not None and threshold > buffer:

# OTHERWISE
i = 0
while (i < len(refList) and threshold > buffer):
    refPoint = refList[i]
    candidates = getCandidateList(refPoint, buffer)
    assessmentDf = evaluateCRSList(candidates, givenPoint, refPoint)
    threshold = assessmentDf.iloc[0]['dist']
    if threshold < bestCRSdf.iloc[0]['dist']:
        bestCRSdf = assessmentDf
        threshold = assessmentDf.iloc[0]['dist']
    i += 1
    
bestCRSdf['dist'] = bestCRSdf['dist'] / 1000
print(bestCRSdf.head(10))

Obtained coordinates for Gent
Prepared list of candidate CRS for [51.054, 3.725]
Calculated distances for candidate CRS for [51.054, 3.725]
           crs                                name         dist
13  epsg:31370           BD72 / Belgian Lambert 72     0.149866
0   epsg:21500  BD50 (Brussels) / Belge Lambert 50     0.227464
12  epsg:31300             BD72 / Belge Lambert 72     0.607695
18   epsg:3447       ETRS89 / Belgian Lambert 2005     1.268719
8   epsg:28992                 Amersfoort / RD New   162.084666
30   epsg:5643                     ED50 / SPBA LCC   317.172994
7   epsg:28991                 Amersfoort / RD Old   385.270250
4   epsg:27561   NTF (Paris) / Lambert Nord France   619.048303
25   epsg:3812       ETRS89 / Belgian Lambert 2008   706.960320
5   epsg:27571        NTF (Paris) / Lambert zone I  1320.117559
